cannon
---

this time not on the fly. Generate data to use each round.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import tensorflow as tf
import keras

In [2]:
pi = np.pi

#simulates a ballistic trajectory and returns the smallest distance to the target position
def simulate_trajectory_distance(speed, azi, pitch, target, origin = np.array([0.,0.,0.]), timestep = 0.1, g = -10.):
    #get cartesian velocity from spherical coordinates
    velocity = speed*np.array([
        np.cos(azi)*np.cos(pitch),
        np.sin(azi)*np.cos(pitch),
        np.sin(pitch)
    ])
#     print(f'Starting velocity {velocity}.')
    #make acceleration vector
    acc = np.array([0.,0., g])
    #record origin and first position after 1 timestep
    history = [origin, origin+ velocity*timestep + (1/2)*acc*timestep**2]
    position = history[-1]
#     print(history)
    #record distances
    distances = [np.linalg.norm(target - origin), np.linalg.norm(target - position)]
    #while the ball is above the ground, timestep to change position
    while position[2] >= 0:
#         print(history[-1])
        position = 2*history[-1] - history[-2] + acc*(timestep**2)
        distances.append(np.linalg.norm(target - position))
        history.append(position)
    return np.array(distances).min()
    

In [47]:
# make data with a model.
#models will take in 3 inputs and spit out 3 outputs. loss should be min distance.


def make_training_data(size, model, field_size = 1000, max_speed = 50, rng = np.random.default_rng(2022)):
    data = []
    for row in range(size):
        target_x, target_y, target_z = field_size*rng.random(3) - field_size/2
        target_z = abs(target_z)
        speed, azi, pitch = model(tf.convert_to_tensor([[target_x,target_y,target_z]]))[0]
        speed1 = float(speed)
        azi1 = float(azi)
        pitch1 = float(pitch)
        speed2 = float(speed*max_speed)
        azi2 = float(azi*2*pi)
        pitch2 = float(pitch*pi)
        distance = simulate_trajectory_distance(speed2,azi2,pitch2, target = [target_x,target_y,target_z])
        data.append([target_x,target_y,target_z, speed1, azi1, pitch1, distance])
    return pd.DataFrame(data, columns = ['target x','target y', 'target z', "speed","azi",'pitch', 'min distance'])

        

In [48]:
make_training_data(5, model)

,target x,target y,target z,speed,azi,pitch,min distance
0,-252.573937,-407.009938,111.763373,0.000000e+00,1.000000,7.482817e-19,491.875731
1,-439.337926,161.033428,255.157778,3.021617e-17,0.001784,0.000000e+00,532.968170
2,-389.131099,-456.944156,85.582526,0.000000e+00,1.000000,0.000000e+00,606.255179
3,488.629256,469.198690,243.028468,1.000000e+00,1.000000,0.000000e+00,716.332191
4,58.762109,-257.652024,177.979709,1.504608e-35,1.000000,1.000000e+00,318.613131


In [49]:
def cannon_loss(output, target):
    speed, azi, pitch = output
    return simulate_trajectory_distance(speed, 2*pi*azi, pi*pitch, target)

In [44]:
#make a model
seed = 2022
rng = np.random.default_rng(seed)

eps = np.finfo(np.float32).eps.item()

gamma = 0.99
rounds_per_load = 5
minimum_acceptable_score = 600
scores_to_average = 10
max_speed = 100 #m/s
field_size = 2000 #m

hit_distance = 1 #1m radius of target
miss_penalty = 500 #add this number of meters to distance as a penalty for missing.

num_inputs = 3 #targetx,y,z
num_outputs = 3 #speed*max_speed, azimuth/(2pi), pitch/(pi)
num_hidden = 32

#make model
inputs = keras.layers.Input(shape = (num_inputs,))
dense_1 = keras.layers.Dense(num_hidden, activation = 'relu')
outputs = keras.layers.Dense(num_outputs, activation = 'sigmoid')
model = keras.Sequential()
model.add(inputs)
model.add(dense_1)
model.add(outputs)
model.compile(optimizer = 'adam', loss = cannon_loss)

In [ ]:
# %%time
xcols = ['target x', 'target y', 'target z']
ycols = ['speed', 'azi', 'pitch']

data = make_training_data(25, model)
train = data[]